In [1]:
import sys, random
sys.path.append('../reconstruction')
sys.path.append('../utils')

from reconstruct_atom_data import *
from pyatom import Atmosphere, Hydrosphere

#create the models
atm_model = Atmosphere()
hyd_model = Hydrosphere()

#load configurations
atm_model.load_config( './config_simon_atm.xml' )
hyd_model.load_config( './config_simon_hyd.xml' )

#get start time, end time and time step from configurations
start_time = atm_model.time_start
end_time = atm_model.time_end
time_step = atm_model.time_step
print('{0}  {1}  {2}'.format(start_time,end_time, time_step))
times = range(start_time, end_time+1, time_step)

#run the models
BATHYMETRY_SUFFIX = 'Ma_Simon.xyz'
for t in range(len(times)):
    time = times[t]
    atm_model.run_time_slice(time)
    hyd_model.run_time_slice(time)
    if t<len(times)-1:
        reconstruct_temperature(time,times[t+1], BATHYMETRY_SUFFIX) 
        reconstruct_precipitation(time,times[t+1], BATHYMETRY_SUFFIX)
        reconstruct_salinity(time,times[t+1], BATHYMETRY_SUFFIX)



0  30  5

   Output is being written to output
   Ma = 0
   bathymetry_path = ../data/topo_grids
   bathymetry_filepath = ../data/topo_grids/0Ma_Simon.xyz


      total number of points at constant hight ... = 64800.000
      number of points on the ocean surface ...... = 42563.000
      number of points on the land surface ....... = 22237.000
      ocean/land ratio ........................... =   1.914

      addition of CO2 by ocean surface ........... =   0.000
      addition of CO2 by land surface ............ =   0.000
      subtraction of CO2 by vegetation ........... =   3.000
      valid for one single point on the surface   



      co2 increase at cretaceous times: ............... co2 increase =   0.000 ppm 
      mean co2 at modern times: ......................... co2 modern = 348.861 ppm 
      mean co2 at cretaceous times: ................... co2 cretaceous = 348.861 ppm 



 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>    2D    <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
 2D AGCM iterational

 3D AGCM iterational process
 max total iteration number nm = 200

 present state of the computation 
 current time slice, number of iterations, maximum and current number of velocity iterations, maximum and current number of pressure iterations 

 Ma = 0     n = 1    velocity_iter_max = 2     velocity_iter = 1    pressure_iter_max = 4    pressure_iter = 1


 3D iterational process for the surface boundary conditions
 printout of maximum and minimum absolute and relative errors of the computed values at their locations: level, latitude, longitude
 residuum: continuity equation ..... =    62.581889   32°N  81°E 11600 m
 dp: pressure Poisson equation ..... =     4.215581   27°N 114°W     0 m
 du: Navier Stokes equation ........ =     0.001684   33°N  70°E  9200 m
 dv: Navier Stokes equation ........ =     0.023603   18°N  69°W     0 m
 dw: Navier Stokes equation ........ =     0.102684   33°N  79°E 11600 m
 dt: energy transport equation ..... =     0.084557   34°N   0°E  5200 m
 dc: vapo

     0 m    0°N 180°E   precipitation .......... =    0.00  mm/d    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2
     0 m    0°N 180°E   Evaporation Penman ..... =    2.96  mm/d    Evaporation Dalton ..... =   10.29  mm/d


 precipitable water average ............ =   47.29    mm    precipitation average per year ........ =    0.00  mm/a    precipitation average per day ......... =    0.00  mm/d
 precipitable water average ............ =   47.29    mm    precipitation NASA average per year ... =  978.27  mm/a    precipitation NASA average per day .... =    2.68  mm/d
 precipitable water average ............ =   47.29    mm    Evaporation_Dalton_average per year ... = 1446.65  mm/a    Evaporation_Dalton_average per day .... =    3.96  mm/d
 co2_average ........................... =  314.26   ppm    Evaporation_Penman_average per year ... =  632.33  mm/a    Evaporation_Penman_average per day .... =    1.7

     0 m   51°N  14°E   Evaporation Penman ..... =    1.34  mm/d    Evaporation Dalton ..... =    1.35  mm/d

 City of Sydney, New South Wales, Australia
     0 m   33°S 151°E   radiation emission...... =  117.04  W/m2    latent heat ............ =    0.00  W/m2    sensible heat .......... =   39.09  W/m2    bottom heat ............ =  -77.95  W/m2
     0 m   33°S 151°E   precipitable water ..... =   52.50    mm    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2
     0 m   33°S 151°E   precipitation .......... =    0.00  mm/d    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2
     0 m   33°S 151°E   Evaporation Penman ..... =    1.69  mm/d    Evaporation Dalton ..... =    0.84  mm/d

 Equator in the central Pacific
     0 m    0°N 180°E   radiation emission...... =  502.40  W/m2    latent heat ............ =    0.00  W/m2    s


 properties of the atmosphere at the surface: 

 max 2D albedo ........... =     0.700000     %   90°N   0°E     0 m           min 2D albedo ........... =     0.150000     %    0°N   1°E     0 m
 max 2D epsilon .......... =     0.010177     %   21°N  72°E     0 m           min 2D epsilon .......... =     0.006937     %   42°N  80°E     0 m
 max 2D topography ....... =  5855.000000     m   28°N  88°E     0 m           min 2D topography ....... =     0.000000     m   90°N   0°E     0 m


 printout of surface data at predefinded locations: level, latitude, longitude

 City of Dresden, Germany, Europe
     0 m   51°N  14°E   radiation emission...... =  380.98  W/m2    latent heat ............ =    0.00  W/m2    sensible heat .......... =   29.99  W/m2    bottom heat ............ = -350.99  W/m2
     0 m   51°N  14°E   precipitable water ..... =   31.95    mm    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2    to fill ................ =    0.00  W/m2


...


 residuum: continuity equation ..... =     0.014433   77°S 156°W
 dp: pressure Poisson equation ..... =     0.000000   84°S 162°W
 dv: Navier Stokes equation ........ =     0.002783   28°S 137°E
 dw: Navier Stokes equation ........ =     0.002199    3°S 104°E




 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>    2D    <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
 2D OGCM iterational process
 max total iteration number nm = 200

 present state of the 2D computation 
  current time slice, number of iterations,                     maximum and current number of velocity iterations, maximum and current number of pressure iterations 

 Ma = 30     n = 9    velocity_iter_max_2D = 2     velocity_iter_2D = 1    pressure_iter_max_2D = 10    pressure_iter_2D = 5


      >>>>>>>>>>>>>>>>>>>>>>>>>>>>>    2D    <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
      2D OGCM iterational process
      max total iteration number nm = 200
      outer pressure loop:  max iteration number pressure_iter_max_2D = 10
      inner velocity loo



 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>    2D    <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
 2D OGCM iterational process
 max total iteration number nm = 200

 present state of the 2D computation 
  current time slice, number of iterations,                     maximum and current number of velocity iterations, maximum and current number of pressure iterations 

 Ma = 30     n = 15    velocity_iter_max_2D = 2     velocity_iter_2D = 1    pressure_iter_max_2D = 10    pressure_iter_2D = 8


      >>>>>>>>>>>>>>>>>>>>>>>>>>>>>    2D    <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
      2D OGCM iterational process
      max total iteration number nm = 200
      outer pressure loop:  max iteration number pressure_iter_max_2D = 10
      inner velocity loop:  max iteration number velocity_iter_max_2D = 2

      n = 15     velocity_iter_2D = 1     pressure_iter_2D = 8     Ma = 30



 2D iterational process for the surface boundary conditions
 printout of maximum and minimum absolute and relative errors of the comp


 Ma = 30     n = 1    velocity_iter_max = 2     velocity_iter = 1    pressure_iter_max = 2    pressure_iter = 1


      >>>>>>>>>>>>>>>>>>>>>>>>>>>>>    3D    <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
      3D OGCM iterational process
      max total iteration number nm = 200
      outer pressure loop:  max iteration number pressure_iter_max = 2
      inner velocity loop:  max iteration number velocity_iter_max = 2

      n = 1     velocity_iter = 1     pressure_iter = 1     Ma = 30



 printout of maximum and minimum absolute and relative errors of the computed values at their locations: level, latitude, longitude

 residuum: continuity equation ..... =     0.000022    0°N 113°W  -550 m
 dp: pressure Poisson equation ..... =  1201.618914   33°N 124°W -1000 m
 du: Navier Stokes equation ........ =     0.000001   20°N  12°W  -975 m
 dv: Navier Stokes equation ........ =     0.147709   32°N 180°E  -600 m
 dw: Navier Stokes equation ........ =     0.098692   32°N 180°E  -550 m
 dt: energy tr


 max salt total .......... =   718.737509   psu   27°S  65°W     0 m           min salt total .......... =    34.050805   psu    3°S  70°E     0 m
 max Salt_Finger ......... =    49.234297   psu   31°S 154°E     0 m           min Salt_Finger ......... =     0.000992   psu   21°N 158°E     0 m
 max Salt_Diffusion ...... =     0.000000   psu   90°N   0°E     0 m           min Salt_Diffusion ...... =     0.000000   psu   87°N 147°E     0 m
 max BuoyancyForce_2D .... =    43.155599     N   87°N 147°E     0 m           min BuoyancyForce_2D .... =     0.000000     N   31°S 154°E     0 m

 deep currents averaged for a two dimensional plane: 

 max upwelling ........... =     0.007580   m/s   10°S  73°W     0 m           min upwelling ........... =     0.000002   m/s    7°S   4°E     0 m
 max downwelling ......... =     0.007575   m/s   13°N 114°E     0 m           min downwelling ......... =     0.000000   m/s   52°N 148°E     0 m
 max bottom water ........ =     0.007580   m/s   10°S  73°W 

 results based on three dimensional considerations of the problem

 max temperature ......... =    39.989160     C   21°N 166°E     0 m           min temperature ......... =   -13.191820     C   38°N 111°W   -50 m
 max 3D u-component ...... =     0.000674  mm/s   84°S 172°W     0 m           min 3D u-component ...... =    -0.000890  mm/s   31°S 123°E     0 m
 max 3D v-component ...... =     0.075087   m/s   33°N  39°E  -400 m           min 3D v-component ...... =    -0.075051   m/s   33°S   8°E  -400 m
 max 3D w-component ...... =     0.071281   m/s    3°N   0°E  -575 m           min 3D w-component ...... =    -0.090609   m/s   33°S  28°E  -550 m
 max pressure dynamic .... =    10.292424   hPa   31°N  58°E     0 m           min pressure dynamic .... =    -8.019904   hPa   20°S  45°E     0 m
 max pressure static ..... =    99.945300   bar   30°S 151°E -1000 m           min pressure static ..... =     0.956088   bar   38°N   0°E     0 m

 salinity based results in the three dimensional s

In [2]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

import create_atm_maps, create_hyd_maps

try:
    topo_dir = '../data/topo_grids/'
    topo_suffix = 'Simon'
    atm_map_output_dir = './atm_maps'
    hyd_map_output_dir = './hyd_maps'

    # v-velocity(m/s), w-velocity(m/s), velocity-mag(m/s), temperature(Celsius), water_vapour(g/kg), 
    # precipitation(mm), precipitable water(mm)
    atm_sub_dirs = ['temperature','v_velocity','w_velocity', 'water_vapour', 'precipitation', 
                'precipitable_water', 'topography', 'velocity']

    create_atm_maps.create_all_maps(atm_sub_dirs, start_time, end_time, time_step, atm_map_output_dir, 
            atm_model.output_path, topo_dir, topo_suffix)

    hyd_sub_dirs = ['temperature','v_velocity','w_velocity', 'salinity', 'bottom_water', 
            'upwelling', 'downwelling', 'velocity']
    
    create_hyd_maps.create_all_maps(hyd_sub_dirs, start_time, end_time, time_step, hyd_map_output_dir,
            atm_model.output_path, topo_dir, topo_suffix)
except:
    import traceback
    traceback.print_exc() 


/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:1352: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Do you want to delete the existing output folder /home/mchin/workspaces/ATOM/mchin/benchmark/atm_maps [Y/N]? y
Delete the existing /home/mchin/workspaces/ATOM/mchin/benchmark/atm_maps
Create a new /home/mchin/workspaces/ATOM/mchin/benchmark/atm_maps



/usr/local/lib/python2.7/dist-packages/numpy/ma/core.py:6442: MaskedArrayFutureWarning: In the future the default for ma.maximum.reduce will be axis=0, not the current None, to match np.maximum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python2.7/dist-packages/numpy/ma/core.py:6442: MaskedArrayFutureWarning: In the future the default for ma.minimum.reduce will be axis=0, not the current None, to match np.minimum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)


./atm_maps/temperature/0_Ma_temperature.png has been saved!
./atm_maps/temperature/5_Ma_temperature.png has been saved!
./atm_maps/temperature/10_Ma_temperature.png has been saved!
./atm_maps/temperature/15_Ma_temperature.png has been saved!
./atm_maps/temperature/20_Ma_temperature.png has been saved!
./atm_maps/temperature/25_Ma_temperature.png has been saved!
./atm_maps/temperature/30_Ma_temperature.png has been saved!
./atm_maps/v_velocity/0_Ma_v_velocity.png has been saved!
./atm_maps/v_velocity/5_Ma_v_velocity.png has been saved!
./atm_maps/v_velocity/10_Ma_v_velocity.png has been saved!
./atm_maps/v_velocity/15_Ma_v_velocity.png has been saved!
./atm_maps/v_velocity/20_Ma_v_velocity.png has been saved!
./atm_maps/v_velocity/25_Ma_v_velocity.png has been saved!
./atm_maps/v_velocity/30_Ma_v_velocity.png has been saved!
./atm_maps/w_velocity/0_Ma_w_velocity.png has been saved!
./atm_maps/w_velocity/5_Ma_w_velocity.png has been saved!
./atm_maps/w_velocity/10_Ma_w_velocity.png has b